# **Importing Modules**

In [ ]:
import cv2
import numpy as np
import skimage.exposure as exposure
from google.colab.patches import cv2_imshow
import os

from google.colab import drive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import shutil

import math
from google.colab.patches import cv2_imshow

import numpy as np
import itertools
import pandas as pd

import json

import os

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import backend as K
import keras
from keras import layers
from tensorflow.keras.optimizers import Adam
from google.colab import files

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt

# **Feature Loading**

In [ ]:
drive.mount('/content/myDrive', force_remount=True)

Mounted at /content/myDrive


In [ ]:
df_s = pd.read_pickle("/content/myDrive/MyDrive/combined-features-1.pkl")

In [ ]:
drive.flush_and_unmount()

In [ ]:
df_s.head()

,tweet_features,image_text_features,image_features,labels
0,"[0.12990479171276093, -0.06545113772153854, 0....","[0.11435211449861526, -0.05234226584434509, 0....","[2.090221643447876, 2.3086254596710205, 0.6487...","[1, 1, 1]"
1,"[0.14545519649982452, -0.09071531891822815, 0....","[0.1072198674082756, -0.07470148056745529, 0.1...","[0.7074947953224182, 0.21232402324676514, 9.40...","[1, 1, 1]"
2,"[0.09893526136875153, -0.004031577613204718, 0...","[0.1072198674082756, -0.07470148056745529, 0.1...","[10.121045112609863, 1.3156002759933472, 0.0, ...","[0, 0, 0]"
3,"[0.07553254812955856, -0.01863664761185646, 0....","[0.1072198674082756, -0.07470148056745529, 0.1...","[3.0665459632873535, 0.6343408823013306, 0.106...","[1, 0, 0]"
4,"[0.053859893232584, 0.04065391421318054, 0.165...","[0.1072198674082756, -0.07470148056745529, 0.1...","[11.037386894226074, 3.784417152404785, 5.7727...","[1, 0, 1]"


In [ ]:
tweet_features = list(df_s["tweet_features"])

In [ ]:
image_text_features = list(df_s["image_text_features"])

In [ ]:
image_features = list(df_s["image_features"])

In [ ]:
labels = list(df_s["labels"])

In [ ]:
int(0.8*(len(tweet_features)))

59878

In [ ]:
X_train_tweet_features = np.array(tweet_features[0:int(0.8*(len(tweet_features)))])

In [ ]:
X_image_text_features = np.array(image_text_features[0:int(0.8*(len(image_text_features)))])

In [ ]:
X_image_features = np.array(image_features[0:int(0.8*(len(image_features)))])

In [ ]:
print(len(X_train_tweet_features))
print(len(X_image_text_features))
print(len(X_image_features))

59878
59878
59878


In [ ]:
[X_train_tweet_features, X_image_text_features, X_image_features]

In [ ]:
y_train = np.array(labels[0:int(0.8*(len(labels)))])

In [ ]:
print(len(y_train))

59878


In [ ]:
print(len(X_train_tweet_features[0]))
print(len(X_image_text_features[0]))
print(len(X_image_features[0]))

768
768
2048


# **Model Building**

In [ ]:
input_text = keras.Input(shape=(768,))
input_image_text = keras.Input(shape=(768,))
input_image = keras.Input(shape=(2048,))


In [ ]:
concatenated = layers.concatenate([input_text, input_image_text, input_image], axis=-1)
output = layers.Dense(3584, activation='relu')(concatenated)
output = layers.Dense(1024, activation='relu')(output)
output = layers.Dense(512, activation='relu')(output)
output = layers.Dense(3, activation='sigmoid')(output)

In [ ]:
model = keras.Model([input_text, input_image_text, input_image], outputs = output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 768)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 768)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 3584)         0           input_1[0][0]                    
                                                                 input_2[0][0]              

In [ ]:
learning_rate = 1e-4
model.compile(optimizer=Adam(learning_rate), loss=['binary_crossentropy', 'binary_crossentropy','binary_crossentropy'])

# **Model Training**

In [ ]:
history = model.fit([X_train_tweet_features, X_image_text_features, X_image_features], y_train, epochs=3,  validation_split= 0.1, batch_size= 1)

Epoch 1/3
53890/53890 [==============================] - 2579s 48ms/step - loss: 0.6357 - val_loss: 0.6091
Epoch 2/3
53890/53890 [==============================] - 2806s 52ms/step - loss: 0.6255 - val_loss: 0.6068
Epoch 3/3
53890/53890 [==============================] - 2834s 53ms/step - loss: 0.6243 - val_loss: 0.6047


In [ ]:
model.save("hatespeech-2.h5")

In [ ]:
files.download("/content/hatespeech-2.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>